In [1]:
from MLPN.loader import init_dataset_train, label2tensor, tensor2label
import os
import torch

from matplotlib import pyplot as plt
import numpy as np


c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
import torch
import torch.nn as nn 
from torch.nn import functional as F
from utils.common import setup_seed 
from MLPN.loader import init_dataset_train
from utils.loader import  init_dataset_test, environments
from utils.metrics import metrics
from MLPN.model import CSWinTransv2_threeIn
from MLPN.utils import extract_feature, get_id, extract_feature, SAM, SupConLoss, one_LPN_output
from torch.autograd import Variable
from torchvision.transforms import InterpolationMode
from torchvision import datasets, transforms

from torch.optim import lr_scheduler
from tqdm.contrib import tzip
from tqdm import tqdm
import numpy as np

import tarfile
import zipfile

from LPN.image_folder_ import CustomData160k_drone, CustomData160k_sat
from utils.competition import get_result_rank10, get_SatId_160k


###########################

import os
import math
from shutil import copyfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.backends.cudnn as cudnn
from torchvision.transforms import InterpolationMode
from torchvision import models

import numpy as np
import random

from utils.common import setup_seed 
from utils.loader import environments, init_dataset_train, tensor2label, label2tensor, init_dataset_test
from torch import nn
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader


class DomainClassifier(nn.Module):
    def __init__(self, domin_num=10) -> None:
        super(DomainClassifier, self).__init__()
        
        # adatped from modern backbone, change last fc to adopt domain
        self.net = models.resnet18(pretrained=True)
        ftr_num = self.net.fc.in_features
        self.net.fc = nn.Linear(ftr_num, domin_num)

    def forward(self, x):
        return self.net(x)

class MLPN_ensemble(nn.Module):
    def __init__(self,
                 replace = ['snow', 'fog', 'fog_rain', 'fog_snow', 'rain_snow'],
                 block = 4
                 ):
        super(MLPN_ensemble, self).__init__()
        self.model_dir = os.path.join(os.getcwd(), 'model', 'MLPNs')

        self.domain_classifier = DomainClassifier()
        self.domain_classifier.load_state_dict(torch.load(os.path.join(self.model_dir, 'DomainClassifier_060.pth')))
        self.domain_classifier.cuda()
        self.domain_classifier.train(False)

        self.ft = {i: CSWinTransv2_threeIn(701, droprate=0.75, decouple=False, infonce=1) for i in environments}
        model_path_default = os.path.join(self.model_dir, 'net_170.pth')
        for style in environments:
            model_path = os.path.join(self.model_dir, '{}_120.pth'.format(style))
            if os.path.isfile(model_path) and style in replace:
                self.ft[style].load_state_dict(torch.load(model_path))
            else:
                self.ft[style].load_state_dict(torch.load(model_path_default))
            
            for i in range(block):
                cls_name = 'classifier'+str(i)
                c = getattr(self.ft[style], cls_name)
                c.classifier = nn.Sequential()

            self.ft[style].cuda()   
            self.ft[style].train(False) 

    def forward(self, x):
        logits = self.domain_classifier(x) 
        pred = tensor2label(logits)
        # print(pred)
        xx = list()
        for i in range(len(pred)):
            feature = self.ft[pred[i]](x[i].unsqueeze(0))
            xx.append(feature[0])
        xxx = torch.cat(xx, 0)
        # print(xxx.shape)
        return [xxx]





class MLPN_ensemble_:
    def __init__(self, 
                 use_wandb=True,
                 wandb_key = '16c9a3f92163ef4df08841029e02fded0cd0cfed') -> None:
        # default parameters
        self.seed = 2024
        self.use_wandb = use_wandb # use wandb to monitor training instead of CLI
        self.wandb_key = wandb_key

        self.pytorch_version =  torch.__version__      
        self.w = 256
        self.h = 256

        # init
        setup_seed(self.seed)
        self.model_dir = os.path.join(os.getcwd(), 'model', 'MLPN')
        os.environ['TORCH_HOME']='./'   


    def test(self, pth=None, query='drone', gallery='satellite', multiple_scale=[1], batchsize=128, style='mixed'):
        # load data
        image_datasets, dataloaders, dataset_sizes = init_dataset_test(batchsize=batchsize, style=style, w=self.w, h=self.h)
        # init label
        gallery_name = 'gallery_' + gallery
        query_name = 'query_' + query 
        gallery_label = get_id(image_datasets[gallery_name].imgs)
        query_label = get_id(image_datasets[query_name].imgs)
        # print(dataset_sizes[gallery_name])
        # load model
        
        model = self.load_model(pth=pth)
        # extract features
        with torch.no_grad():
            query_feature = extract_feature(model,dataloaders[query_name], view=query, ms=multiple_scale)
            gallery_feature = extract_feature(model,dataloaders[gallery_name], view=gallery, ms=multiple_scale)

        # calculate 
        m = metrics(query_feature, query_label, gallery_feature, gallery_label)
        print("Recall@1: {:.2f}".format(m[0]))
        print("Recall@5: {:.2f}".format(m[1]))
        print("Recall@10: {:.2f}".format(m[2]))
        print("Recall@top1: {:.2f}".format(m[3]))
        print("Recall@AP: {:.2f}".format(m[4]))
        # return m

    def get_competition_submit(self, data160k_dir='D://dataset/university-160k-wx', save_file = 'answer.txt', pth=None, multiple_scale=[1], batchsize=128):
        # a part of test set of UAVM'24 competition is provided as tar file
        if not os.path.exists(os.path.join(data160k_dir, 'gallery_satellite_160k')):
            tar_file = os.path.join(data160k_dir, 'gallery_satellite_160k.tar.gz')
            if os.path.isfile(tar_file):
                print('Found dataset tar file. Extracting...')
                with tarfile.open(tar_file, 'r:gz') as tar:
                    tar.extractall(path=data160k_dir)
                print('Extract done')
        else:
            print('Found dataset')

        query_name = os.path.join(data160k_dir, 'query_drone_name.txt')
        if os.path.isfile(save_file):
            os.remove(save_file) 
            os.remove('answer.zip')
        results_rank10 = [] 

        data_transforms = transforms.Compose([
            transforms.Resize((self.h, self.w), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        image_datasets = {}
        image_datasets['gallery_satellite_160k'] = CustomData160k_sat(os.path.join(data160k_dir, 'gallery_satellite_160k'), data_transforms)
        image_datasets['query_drone_160k'] = CustomData160k_drone( os.path.join(data160k_dir, 'query_drone160k_wx') ,data_transforms, query_name = query_name)

        dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batchsize,
                                                        shuffle=False, num_workers=16) for x in
                        ['gallery_satellite_160k','query_drone_160k']}
        
        gallery_path = image_datasets['gallery_satellite_160k'].imgs
        gallery_label, gallery_path  = get_SatId_160k(gallery_path)
        
        # load model
        # model = self.load_model(pth=pth)
        model = MLPN_ensemble()

        # Extract features
        with torch.no_grad():
            query_feature = extract_feature(model,dataloaders['query_drone_160k'], view='drone', ms=multiple_scale, testing=True)
            gallery_feature = extract_feature(model,dataloaders['gallery_satellite_160k'], view='satellite', ms=multiple_scale)
        
        query_feature = query_feature.cuda()
        gallery_feature = gallery_feature.cuda()

        gallery_label = np.array(gallery_label)
        for i in tqdm(range(len(query_feature)), desc='Evaluate Rank10 results'):
            result_rank10 = get_result_rank10(query_feature[i], gallery_feature, gallery_label)
            results_rank10.append(result_rank10)
            
        results_rank10 = np.row_stack(results_rank10)
        with open(save_file, 'w') as f:
            for row in results_rank10:
                f.write('\t'.join(map(str, row)) + '\n')

        # zip
        zip_name = os.path.join(os.getcwd(), 'answer.zip')
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            zipf.write(save_file, save_file)

        

c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = MLPN_ensemble_()


In [3]:
m.get_competition_submit()

Found dataset


c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\lyq\.conda\envs\pytorch\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: i

In [4]:
m = MLPN_()
m.batchsize=128
m.test()

951
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:12<00:00, 3013.90it/s]


Recall@1: 91.75
Recall@5: 97.59
Recall@10: 98.44
Recall@top1: 98.54
Recall@AP: 93.05


(tensor(91.7475),
 tensor(97.5882),
 tensor(98.4361),
 tensor(98.5418),
 93.05280791725373)

In [11]:
m = MLPN_()
m.batchsize=128

from utils.loader import environments 

for style in environments:
    print(style)
    m.test(style=style, pth='net_170')

normal
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2883.25it/s]


Recall@1: 94.07
Recall@5: 98.34
Recall@10: 98.79
Recall@top1: 98.86
Recall@AP: 95.03
dark
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:12<00:00, 2924.97it/s]


Recall@1: 93.92
Recall@5: 98.32
Recall@10: 98.85
Recall@top1: 98.90
Recall@AP: 94.91
fog
load model: net_170.pth


...

Recall@1: 91.23
Recall@5: 97.36
Recall@10: 98.33
Recall@top1: 98.40
Recall@AP: 92.61
light
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2888.61it/s]


Recall@1: 90.40
Recall@5: 97.35
Recall@10: 98.34
Recall@top1: 98.44
Recall@AP: 91.96
wind
load model: net_170.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:13<00:00, 2909.69it/s]

Recall@1: 89.58
Recall@5: 96.65
Recall@10: 97.87
Recall@top1: 98.00
Recall@AP: 91.17


trained on original dataset

In [10]:
for style in environments:
    print(style)
    m.test(style=style, pth='net_170_')

normal
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2490.53it/s]


Recall@1: 93.71
Recall@5: 97.87
Recall@10: 98.54
Recall@top1: 98.60
Recall@AP: 94.69
dark
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2453.34it/s]


Recall@1: 92.21
Recall@5: 97.69
Recall@10: 98.48
Recall@top1: 98.57
Recall@AP: 93.45
fog
load model: net_170_.pth


...

Recall@1: 37.43
Recall@5: 56.25
Recall@10: 64.51
Recall@top1: 65.67
Recall@AP: 41.99
light
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2402.24it/s]


Recall@1: 85.42
Recall@5: 95.12
Recall@10: 97.03
Recall@top1: 97.24
Recall@AP: 87.59
wind
load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:15<00:00, 2452.96it/s]

Recall@1: 78.20
Recall@5: 91.55
Recall@10: 94.54
Recall@top1: 94.82
Recall@AP: 81.21


In [13]:
m.test(style='mixed', pth='net_170_')

load model: net_170_.pth


Evaluate metrics: 100%|██████████| 37855/37855 [00:14<00:00, 2526.40it/s]


Recall@1: 63.14
Recall@5: 76.16
Recall@10: 80.78
Recall@top1: 81.38
Recall@AP: 66.21
